# Musicians- Medium

In [1]:
# Prerequesites
from pyhive import hive
%load_ext sql
%sql hive://cloudera@quickstart.cloudera:10000/sqlzoo
%config SqlMagic.displaylimit = 20

## 6.
**List the names, dates of birth and the instrument played of living musicians who play a instrument which Theo also plays.**

In [3]:
%%sql
WITH ins AS (
  SELECT instrument FROM 
    musician JOIN performer ON (
        musician.m_no=performer.perf_is) 
    WHERE m_name LIKE 'Theo%'
)
SELECT m_name, born, performer.instrument
  FROM musician JOIN performer ON (
      musician.m_no=performer.perf_is) JOIN ins ON
    (ins.instrument=performer.instrument)
    WHERE died IS NULL AND m_name NOT LIKE 'Theo%'
    ORDER BY m_name

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


m_name,born,instrument
Harry Forte,1951-02-28,drums
Harry Forte,1951-02-28,violin
James First,1965-06-10,violin
Jeff Dawn,1945-12-12,violin
John Smith,1950-03-03,violin


## 7.
**List the name and the number of players for the band whose number of players is greater than the average number of players in each band.**

In [4]:
%%sql
WITH t AS (
    SELECT DISTINCT band_name, perf_is
      FROM band JOIN plays_in ON (
          band.band_no=plays_in.band_id) JOIN performer ON (
          performer.perf_no=plays_in.player)
), summ AS (
    SELECT band_name, COUNT(*) nmbr 
        FROM t GROUP BY band_name
)
SELECT summ.band_name, summ.nmbr
    FROM summ JOIN (SELECT AVG(nmbr) mean FROM summ) a
    WHERE summ.nmbr>a.mean

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


band_name,nmbr
AASO,7
Oh well,5
ROP,7


## 8.
**List the names of musicians who both conduct and compose and live in Britain.**

In [5]:
%%sql
SELECT DISTINCT m_name FROM
  musician JOIN composer ON (
       musician.m_no=composer.comp_is) JOIN
    place ON (musician.living_in=place.place_no) JOIN
    performance ON (
        performance.conducted_by=musician.m_no) 
    WHERE place_country IN ('England', 'Scotland')
    ORDER BY m_name

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


m_name
Fred Bloggs
Jeff Dawn
Phil Hot
Rose Spring
Tony Smythe


## 9.
**Show the least commonly played instrument and the number of musicians who play it.**

In [8]:
%%sql
WITH t AS (
    SELECT instrument, COUNT(*) n
      FROM performer JOIN plays_in ON (
          performer.perf_no=plays_in.player) JOIN
        performance ON (performance.gave=plays_in.band_id)
        GROUP By instrument
        ORDER BY n
        LIMIT 1
)
SELECT performer.instrument, COUNT(*) n_player
  FROM performer JOIN t ON (performer.instrument=t.instrument)
    GROUP BY performer.instrument

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


instrument,n_player
clarinet,1


## 10.
**List the bands that have played music composed by Sue Little; Give the titles of the composition in each case.**

In [9]:
%%sql
WITH t AS (
  SELECT c_no, c_title
    FROM composition JOIN has_composed ON (
        composition.c_no=has_composed.cmpn_no) JOIN
    composer ON (composer.comp_no=has_composed.cmpr_no) JOIN
    musician ON (musician.m_no=composer.comp_is)
    WHERE m_name='Sue Little'
)
SELECT band_name, c_title
  FROM t JOIN performance ON (t.c_no=performance.performed) JOIN
    band ON (performance.gave=band.band_no)
    ORDER BY band_name

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


band_name,c_title
BBSO,Slow Symphony Blowing
BBSO,Slow Song
Somebody Loves this,Slow Symphony Blowing
Swinging strings,Slow Song
The left Overs,Slow Song
